In [1]:
from dash import Dash, html, dcc, Input, Output, callback
import pandas as pd
import plotly.express as px
import scipy.stats as stats
import plotly.io as po
import plotly.graph_objs as go

In [2]:
# data load
ice_data ='https://raw.githubusercontent.com/lhshs/mydataset/main/genia/2307_NLP/ice_model.csv'
tsh_data = 'https://raw.githubusercontent.com/lhshs/mydataset/main/genia/2307_NLP/tsh_model.csv'
ice = pd.read_csv(ice_data, index_col=0)
tsh = pd.read_csv(tsh_data, index_col=0)
# df_boxsstrip = pd.read_csv(boxstrip_df)#.set_index('지역')

In [3]:
ice.columns

Index(['text', 'score',
       'jaehyeong/koelectra-base-v3-generalized-sentiment-analysis',
       'klue/bert-base', 'matthewburke/korean_sentiment',
       'monologg/koelectra-base-v2-discriminator', 'skt/kogpt2-base-v2',
       'beomi/kcbert-large'],
      dtype='object')

In [9]:
ice['jaehyeong/koelectra-base-v3-generalized-sentiment-analysis'].value_counts()

 1    5100
-1      77
 0      64
Name: jaehyeong/koelectra-base-v3-generalized-sentiment-analysis, dtype: int64

In [46]:
ice

,text,score,jaehyeong/koelectra-base-v3-generalized-sentiment-analysis,klue/bert-base,matthewburke/korean_sentiment,monologg/koelectra-base-v2-discriminator,skt/kogpt2-base-v2,beomi/kcbert-large
0,알차네요 무엇보다도 빨리들을 수 있어 좋습니다,1,1,0,1,0,0,0
1,명불허전 명강사라 그런지 말씀을 참 잘하시네요~,1,1,0,1,0,0,0
2,강추 30년 가까이 일하면서도 놓치고 있었던 것들을 알려주었습니다. 그리고 무조건 ...,1,1,0,1,0,0,0
3,패들렛과 띵커벨 활용 잘 할 수 있어요 패들렛과 띵커벨 활용하고 있었으나 다양한 교...,1,1,0,1,0,0,1
4,"필수 연수 부담 없이 편안하게 안전 연수 올 해 꼭 받아야 하는데, 부담 없이 편안...",1,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...
5236,알베르토가 함께하는 핵심역량 연계 다문화 교육 여타 연수와 다르게 새로 알게된 내용...,1,1,0,1,0,0,0
5237,전문가가 알려주는 창의 영재교육의 비법 연수 후기 영재 교육에 대한 틀을 잡아주시는...,1,1,0,1,0,0,0
5238,도도한 교사생활 연수를 통해 자신감을 더 갖게 되었다.,1,1,0,1,0,0,0
5239,교실에서 활용할 수 있는 앱들 중심으로 재미있게 들었어요. 교실에서 활용할 수 있는...,1,1,0,1,0,0,0


,beomi/kcbert-large,score
0,0,1
1,0,1
2,0,1
3,1,1
4,0,1
...,...,...
5236,0,1
5237,0,1
5238,0,1
5239,0,1


In [175]:
# Run this app with `python app.py` and
# visit http://127.0.0.1:8050/ in your web browser.

app = Dash(__name__)

# Dropdown options setup
dropdown_options = [{'label': col, 'value': col} for col in ice.iloc[:,2:].columns]

app.layout = html.Div(children=[
    html.H2(children='Score & Predict by Pre-Trained Model'),
    html.Div(children='''
             Select Model.
             '''),

    html.Div([
        dcc.Dropdown(
                id='column-dropdown',
                options=dropdown_options,
                value=ice.columns[-1]
            ),
        dcc.Graph(id='pie-chart-container')
        ]),

    # html.Div(
    #         dcc.Graph(
    #             figure={
    #                 'data': [
    #                     go.Pie(
    #                         labels=ice['score'].unique(),
    #                         values=ice['score'].value_counts(),
    #                         marker=dict(colors=ice['score'].unique()),
    #                         # textinfo='label+percent'
    #                             )   
    #                         ],
    #                         'layout': go.Layout(
    #                             title='Score',
    #                             # xaxis={'-1':'negative', '0':'Neutral', '1':'Positive'},
    #                             plot_bgcolor='rgba(0,0,0,0)',
    #                             paper_bgcolor='rgba(0,0,0,0)',
    #                             autosize=False,
    #                                             )
    #                     }
    #                 )
                
    #         )
])


# Callback to update the pie chart based on dropdown selection
@callback(
    Output('pie-chart-container', 'children'),
    [Input('column-dropdown', 'value')]
)
def update_pie_chart(selected_column):
    return ice_pie_chart(selected_column)


# Create a pie chart using Plotly
def ice_pie_chart(selected_column):
    pie_chart = dcc.Graph(
        figure={
            'data': [
                go.Pie(
                    labels=ice[selected_column].unique(),
                    values=ice[selected_column].value_counts(),
                    marker=dict(colors=ice[selected_column].unique()),
                    # textinfo='label+percent'
                      )
            ],
            'layout': go.Layout(
                title='Ice Score & Predict by Pre-Trained Model',
                # xaxis={'-1':'negative', '0':'Neutral', '1':'Positive'},
                plot_bgcolor='rgba(0,0,0,0)',
                paper_bgcolor='rgba(0,0,0,0)',
                autosize=True,
            )
        }
    )
    return pie_chart


if __name__ == '__main__':
    app.run(debug=True)